<a href="https://colab.research.google.com/github/HyeJin816/ESAA_22/blob/main/0523_ch9%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **9.8 파이썬추천시스템패키지**


## **Surprise 패키지**

: 파이썬 기반 추천 시스템 패키지
> 사용자&아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반 잠재 요인 협업 필터링  
> 사이킷런의 핵심 API와 유사  
> 모델 셀렉션, 평가, 하이퍼 파라미터 튜닝 등의 기능을 제공

In [2]:
pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 23.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633743 sha256=221ce18b0d660dd1fae25f86a95be430fef577b8b6e975d9ae38f4a14cb87786
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
import surprise 

print(surprise.__version__)

1.1.1


## **Surprise 를 이용한 추천 시스템 구축**

In [4]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [5]:
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [6]:
algo = SVD()
algo.fit(trainset) 

**Surprise에서 추천 예측 메서드**
* ```test()```: 사용자-아이템 평점 데이터 세트 전체에 대한 추천 데이터 추천 세트
* ```prdict()```: 개별 사용자와 영화에 대한 추천 평점 반환

In [7]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.571907616186998, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.980445787254438, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.0419942500203785, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.4797829745835727, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.195116428806458, details={'was_impossible': False})]

*'was_impossible'*
> 예측 불가능 데이터 True  
> 예측 가능 데이터 False

In [8]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.571907616186998),
 ('882', '291', 3.980445787254438),
 ('535', '507', 4.0419942500203785)]

In [9]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.34   {'was_impossible': False}


In [10]:
accuracy.rmse(predictions)

RMSE: 0.9462


0.9461770940293464

## **Surprise 주요 모듈 소개**

데이터 세트의 칼럼 순서가 *사용자 아이디 - 아이템 아이디 - 평점* 순  
> ```Dataset.load_builtin(name='ml-100k')```: 무비렌즈 FTP 서버에서 받은 데이터를 저장  
> ```Dataset.load_from_file(file_path, reader)``` : OS파일에서 데이터 로딩. 
> ```Dataset.load_from_df(df, reader)``` : 판다스 DataFrame에서 데이터 로딩

다른 OS 파일 사용 위한 Reader 클래스 생성 파라미터
* ```line_format(string)```: 칼럼을 순서대로 나열. 문자열을 공백으로 분리해 칼럼으로 인식
* ```sep(char)```: 칼럼을 분리하는 분리자. 디폴트는 ```\t```. (DataFrame에서 입력받을 경우에는 기재할 필요없음)
* ```rating_scale(Tuple, optional)``` : 평점 값의 최소~최대 설정

In [29]:
ratings = pd.read_csv('/content/ratings.csv')
#ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('/content/ratings_noh.csv', index=False, header=False)

In [30]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('/content/ratings_noh.csv',reader=reader)

In [31]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8583


0.8583461095509001

### **판다스 DataFrame에서 Surprise 데이터 세트로 로딩**

In [34]:
ratings = pd.read_csv('/content/ratings.csv')
reader = Reader(rating_scale = (0.5, 5.0))

#ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size = .25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predidctions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8583


0.8583461095509001

### **Surprise 추천 알고리즘 클래스**

> **SVD**: 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘  
> **KNNBasic**: 최근접 이웃 협업 필터링을 위한 KNN 알고리즘  
> **BaselineOnly**: 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘  

베이스라인 평점 : 개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영하여 평점 부과

## **교차 검증과 하이퍼 파라미터 튜닝**

In [35]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('/content/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8537  0.8524  0.8547  0.8539  0.8563  0.8542  0.0013  
MAE (testset)     0.6517  0.6491  0.6513  0.6513  0.6526  0.6512  0.0012  
Fit time          31.92   30.95   31.36   31.16   31.03   31.28   0.35    
Test time         1.32    1.32    1.35    1.38    1.37    1.35    0.03    


{'fit_time': (31.92136836051941,
  30.948162317276,
  31.357829809188843,
  31.16181182861328,
  31.034483432769775),
 'test_mae': array([0.65172179, 0.64905722, 0.65129049, 0.65133695, 0.65259563]),
 'test_rmse': array([0.85368576, 0.85236282, 0.85472251, 0.85390333, 0.85627741]),
 'test_time': (1.3217451572418213,
  1.315734624862671,
  1.3538198471069336,
  1.3846731185913086,
  1.3718748092651367)}

In [36]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8633195466466272
{'n_epochs': 20, 'n_factors': 50}


## **Surprise를 이용한 개인화 영화 추천 시스템 구축**

In [39]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='/content/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [40]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [41]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('/content/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [42]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 2.91   {'was_impossible': False}


In [43]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 1 추천대상 영화수: 9741 전체 영화수: 9742


In [44]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    #알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행 수 결과를 list로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    #predictions list 객체는 surprise의 Predictions 객체를 원소로 가짐
    #이를 est 값으로 정렬하기 위해 아래의 sortkey_est 함수 정의
    #sortkey_est 함수는 list 객체의 sort() 함수의 키 값을 사용되어 정렬 수행
    def sortkey_est(pred):
        return pred.est

    #sortkey_esr() 반환값의 내림차순으로 정렬 수행 후 top_n개의 최상위 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions=predictions[:top_n]

    #top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

In [45]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('#### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
    print(top_movie[1], ':', round(top_movie[2], 3))

평점 매긴 영화수: 1 추천대상 영화수: 9741 전체 영화수: 9742
#### Top-10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.654
Pulp Fiction (1994) : 4.62
Shawshank Redemption, The (1994) : 4.556
Cinema Paradiso (Nuovo cinema Paradiso) (1989) : 4.556
Life Is Beautiful (La Vita è bella) (1997) : 4.526
Wages of Fear, The (Salaire de la peur, Le) (1953) : 4.507
High and Low (Tengoku to jigoku) (1963) : 4.501
Jetée, La (1962) : 4.467
Voices from the List (2004) : 4.448
Black Mirror: White Christmas (2014) : 4.441
